# preamble

In [1]:
import functools as ft
import graphviz as gv
import itertools
import numpy as np
import operator
import os
import seaborn as sns
import sys

In [2]:
def insert_into_path(fo):
    for fo_in_path in sys.path:
        if fo == fo_in_path:
            print('noop')
            return
        
    sys.path.insert(0, fo)
    
insert_into_path('../h2py')
insert_into_path('../h2py/chromatic')
insert_into_path('../../python')

In [3]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [4]:
from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1>'))

# exact

## preamble

In [5]:
from generic.ugraph import UGraph
from generic.graph_utils import GraphUtils
from generic.graph_types import GraphTypes
from h2py.chromatic import BruteForceChromaticNumber
from h2py.chromatic import BruteForceChromaticIndex

from h2py.generic.unique_certificates import UniqueCertificates

In [6]:
def csv_to_ci_inputs():
    fi_unique_certificates = '../../data/db_unique_certificates.csv'
    if not os.path.isfile(fi_unique_certificates):
        return []
    
    lines = []
    with open(fi_unique_certificates, 'r') as f:
        for line in f:
            if len(line) < 2:
                continue
            num_vertices, graph_id = line.split(",")
            num_vertices = int(num_vertices)
            graph_id = int(graph_id)
            lines.append((num_vertices, graph_id))
    return lines

## vertex coloring

In [15]:
ci_pmtrs = csv_to_ci_inputs()

@interact(ix_pmtr=range(len(ci_pmtrs)))
def show_coloration(ix_pmtr):
    num_vertices, graph_id = ci_pmtrs[ix_pmtr]
    graph = UGraph.graph_from_graph_id(num_vertices, graph_id)
    coloration = BruteForceChromaticNumber.compute(graph)
    gviz = GraphUtils.color_vertices(graph, coloration)
    display(HTML(gviz._repr_svg_()))

# interact(show_coloration, pmtrs=ci_pmtrs)

interactive(children=(Dropdown(description='ix_pmtr', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28), value=0), Output()), _dom_classes=('widget-interact',))

## edge coloring

In [16]:
ci_pmtrs = csv_to_ci_inputs()

@interact(ix_pmtr=range(len(ci_pmtrs)))
def show_coloration(ix_pmtr):
    num_vertices, graph_id = ci_pmtrs[ix_pmtr]
    graph = UGraph.graph_from_graph_id(num_vertices, graph_id)
    coloration_edges = BruteForceChromaticIndex.compute(graph)
    print(coloration_edges)
    gviz = GraphUtils.color_edges(graph, coloration_edges)
    display(HTML(gviz._repr_svg_()))

# interact(show_coloration, pmtrs=ci_pmtrs)

interactive(children=(Dropdown(description='ix_pmtr', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28), value=0), Output()), _dom_classes=('widget-interact',))

## dominating set

In [ ]:
from h2py.domination.brute_force_dom_number import BruteForceDomNumber

In [ ]:
graphs = list(UniqueCertificates("../../data/db_unique_certificates.csv"))

@interact(graph=graphs)
def show_dominating_set(graph):
    dom_set = BruteForceDomNumber(graph).compute()
    gviz = GraphUtils.color_dominating_set(graph, dom_set)
    display(HTML(gviz._repr_svg_()))

## edge connectivity

In [7]:
from h2py.edge_connectivity.brute_force_edge_connectivity import BruteForceEdgeConnectivity

In [11]:
graphs = list(UniqueCertificates("../../data/db_unique_certificates.csv"))

@interact(graph=graphs[2:3])
def show_bridges(graph):
    bridges = BruteForceEdgeConnectivity(graph).compute()
    print(bridges)
    gviz = GraphUtils.color_edge_subset(graph, bridges)
    display(HTML(gviz._repr_svg_()))

interactive(children=(Dropdown(description='graph', options=(<h2py.generic.ugraph.UGraph object at 0x7f2000d6f7b8>,), value=<h2py.generic.ugraph.UGraph object at 0x7f2000d6f7b8>), Output()), _dom_classes=('widget-interact',))

## tarjen

In [ ]:
from h2py.generic.tarjen_bridges import TarjenBridges

In [ ]:
graphs = list(UniqueCertificates("../../data/db_unique_certificates.csv"))

@interact(graph=graphs)
def show_bridges(graph):
    bridges = TarjenBridges(graph).compute()
    gviz = GraphUtils.color_edge_subset(graph, bridges)
    display(HTML(gviz._repr_svg_()))

## longest path

In [13]:
from h2py.path.longest_path_brute_force import LongestPathBruteForce

In [14]:
LongestPathBruteForce.find_longest_subpath(graphs[0], [0, 1, 2])

NameError: name 'graphs' is not defined

In [20]:
def label_path(cls, graph: GraphTypes.UGraph, path: GraphTypes.Vertices, engine='neato'):
    graph_viz_object = gv.Graph(engine=engine)
    color_xkcd = sns.xkcd_palette(np.random.choice(list(sns.xkcd_rgb.keys()), 1))[0]
    hex_color = cls.sns_color_to_hex(color_xkcd)
    edge_set = set(GraphUtils.make_edge_tuple(*edge) for edge in graph.edges())
    
    for ix_edge, (node1, node2) in enumerate(zip(path[:-1], path[1:])):
        graph_viz_object.edge(str(node1), str(node2), label=str(ix_edge), color=hex_color, penwidth=str(4), fontsize=str(10))
        edge_set.difference_update({GraphUtils.make_edge_tuple(node1, node2)})
        
    for col, row in edge_set:
        graph_viz_object.edge(str(col), str(row), fontsize=str(10), penwidth=str(2))
    return graph_viz_object

In [21]:
graphs = list(UniqueCertificates("../../data/db_unique_certificates.csv"))

@interact(graph=graphs)
def show_bridges(graph):
    path = LongestPathBruteForce.compute(graph)
    print(path)
    gviz = label_path(GraphUtils, graph, path)
    display(HTML(gviz._repr_svg_()))

interactive(children=(Dropdown(description='graph', options=(<generic.ugraph.UGraph object at 0x7fa5722bb828>, <generic.ugraph.UGraph object at 0x7fa5722bb7f0>, <generic.ugraph.UGraph object at 0x7fa5722bb860>, <generic.ugraph.UGraph object at 0x7fa5722bb898>, <generic.ugraph.UGraph object at 0x7fa5722bb8d0>, <generic.ugraph.UGraph object at 0x7fa5722bb908>, <generic.ugraph.UGraph object at 0x7fa5722bb940>, <generic.ugraph.UGraph object at 0x7fa5722bb978>, <generic.ugraph.UGraph object at 0x7fa5722bb9b0>, <generic.ugraph.UGraph object at 0x7fa5722bb9e8>, <generic.ugraph.UGraph object at 0x7fa5722bba20>, <generic.ugraph.UGraph object at 0x7fa5722bba58>, <generic.ugraph.UGraph object at 0x7fa5722bba90>, <generic.ugraph.UGraph object at 0x7fa5722bbac8>, <generic.ugraph.UGraph object at 0x7fa5722bbb00>, <generic.ugraph.UGraph object at 0x7fa5722bbb38>, <generic.ugraph.UGraph object at 0x7fa5722bbb70>, <generic.ugraph.UGraph object at 0x7fa5722bbba8>, <generic.ugraph.UGraph object at 0x7fa5722bbbe0>, <generic.ugraph.UGraph object at 0x7fa5722bbc18>, <generic.ugraph.UGraph object at 0x7fa5722bbc50>, <generic.ugraph.UGraph object at 0x7fa5722bbc88>, <generic.ugraph.UGraph object at 0x7fa5722bbcc0>, <generic.ugraph.UGraph object at 0x7fa5722bbcf8>, <generic.ugraph.UGraph object at 0x7fa5722bbd30>, <generic.ugraph.UGraph object at 0x7fa5722bbd68>, <generic.ugraph.UGraph object at 0x7fa5722bbda0>, <generic.ugraph.UGraph object at 0x7fa5722bbdd8>, <generic.ugraph.UGraph object at 0x7fa5722bbe10>, <generic.ugraph.UGraph object at 0x7fa5722bbe48>, <generic.ugraph.UGraph object at 0x7fa5722bbe80>, <generic.ugraph.UGraph object at 0x7fa5722bbeb8>, <generic.ugraph.UGraph object at 0x7fa5722bbef0>, <generic.ugraph.UGraph object at 0x7fa5722bbf28>, <generic.ugraph.UGraph object at 0x7fa5722bbf60>, <generic.ugraph.UGraph object at 0x7fa5722bbf98>, <generic.ugraph.UGraph object at 0x7fa5722bbfd0>, <generic.ugraph.UGraph object at 0x7fa5722bf048>, <generic.ugraph.UGraph object at 0x7fa5722bf080>, <generic.ugraph.UGraph object at 0x7fa5722bf0b8>, <generic.ugraph.UGraph object at 0x7fa5722bf0f0>, <generic.ugraph.UGraph object at 0x7fa5722bf128>, <generic.ugraph.UGraph object at 0x7fa5722bf160>, <generic.ugraph.UGraph object at 0x7fa5722bf198>, <generic.ugraph.UGraph object at 0x7fa5722bf1d0>, <generic.ugraph.UGraph object at 0x7fa5722bf208>, <generic.ugraph.UGraph object at 0x7fa5722bf240>, <generic.ugraph.UGraph object at 0x7fa5722bf278>, <generic.ugraph.UGraph object at 0x7fa5722bf2b0>, <generic.ugraph.UGraph object at 0x7fa5722bf2e8>, <generic.ugraph.UGraph object at 0x7fa5722bf320>, <generic.ugraph.UGraph object at 0x7fa5722bf358>, <generic.ugraph.UGraph object at 0x7fa5722bf390>, <generic.ugraph.UGraph object at 0x7fa5722bf3c8>, <generic.ugraph.UGraph object at 0x7fa5722bf400>, <generic.ugraph.UGraph object at 0x7fa5722bf438>, <generic.ugraph.UGraph object at 0x7fa5722bf470>, <generic.ugraph.UGraph object at 0x7fa5722bf4a8>, <generic.ugraph.UGraph object at 0x7fa5722bf4e0>, <generic.ugraph.UGraph object at 0x7fa5722bf518>, <generic.ugraph.UGraph object at 0x7fa5722bf550>, <generic.ugraph.UGraph object at 0x7fa5722bf588>, <generic.ugraph.UGraph object at 0x7fa5722bf5c0>, <generic.ugraph.UGraph object at 0x7fa5722bf5f8>, <generic.ugraph.UGraph object at 0x7fa5722bf630>, <generic.ugraph.UGraph object at 0x7fa5722bf668>, <generic.ugraph.UGraph object at 0x7fa5722bf6a0>, <generic.ugraph.UGraph object at 0x7fa5722bf6d8>, <generic.ugraph.UGraph object at 0x7fa5722bf710>, <generic.ugraph.UGraph object at 0x7fa5722bf748>, <generic.ugraph.UGraph object at 0x7fa5722bf780>, <generic.ugraph.UGraph object at 0x7fa5722bf7b8>, <generic.ugraph.UGraph object at 0x7fa5722bf7f0>, <generic.ugraph.UGraph object at 0x7fa5722bf828>, <generic.ugraph.UGraph object at 0x7fa5722bf860>, <generic.ugraph.UGraph object at 0x7fa5722bf898>, <generic.ugraph.UGraph object at 0x7fa5722bf8d0>, <generic.ugraph.UGraph object at 0x7fa5722bf908>, <generic.ugraph.UGraph object at 0x7fa5

## maximum matching

In [13]:
from h2py.matching.maximum_matching_brute_force import MaximumMatchingBruteForce

In [14]:
graphs = list(UniqueCertificates("../../data/db_unique_certificates.csv"))

@interact(graph=graphs)
def show_bridges(graph):
    edges = MaximumMatchingBruteForce.compute(graph)
    gviz = GraphUtils.color_edge_subset(graph, edges)
    display(HTML(gviz._repr_svg_()))

interactive(children=(Dropdown(description='graph', options=(<generic.ugraph.UGraph object at 0x7f88d382e6d8>, <generic.ugraph.UGraph object at 0x7f88d382e6a0>, <generic.ugraph.UGraph object at 0x7f88d382e710>, <generic.ugraph.UGraph object at 0x7f88d382e748>, <generic.ugraph.UGraph object at 0x7f88d382e780>, <generic.ugraph.UGraph object at 0x7f88d382e7b8>, <generic.ugraph.UGraph object at 0x7f88d382e7f0>, <generic.ugraph.UGraph object at 0x7f88d382e828>, <generic.ugraph.UGraph object at 0x7f88d382e860>, <generic.ugraph.UGraph object at 0x7f88d382e898>, <generic.ugraph.UGraph object at 0x7f88d382e8d0>, <generic.ugraph.UGraph object at 0x7f88d382e908>, <generic.ugraph.UGraph object at 0x7f88d382e940>, <generic.ugraph.UGraph object at 0x7f88d382e978>, <generic.ugraph.UGraph object at 0x7f88d382e9b0>, <generic.ugraph.UGraph object at 0x7f88d382e9e8>, <generic.ugraph.UGraph object at 0x7f88d382ea20>, <generic.ugraph.UGraph object at 0x7f88d382ea58>, <generic.ugraph.UGraph object at 0x7f88d382ea90>, <generic.ugraph.UGraph object at 0x7f88d382eac8>, <generic.ugraph.UGraph object at 0x7f88d382eb00>, <generic.ugraph.UGraph object at 0x7f88d382eb38>, <generic.ugraph.UGraph object at 0x7f88d382eb70>, <generic.ugraph.UGraph object at 0x7f88d382eba8>, <generic.ugraph.UGraph object at 0x7f88d382ebe0>, <generic.ugraph.UGraph object at 0x7f88d382ec18>, <generic.ugraph.UGraph object at 0x7f88d382ec50>, <generic.ugraph.UGraph object at 0x7f88d382ec88>, <generic.ugraph.UGraph object at 0x7f88d382ecc0>, <generic.ugraph.UGraph object at 0x7f88d382ecf8>, <generic.ugraph.UGraph object at 0x7f88d382ed30>, <generic.ugraph.UGraph object at 0x7f88d382ed68>, <generic.ugraph.UGraph object at 0x7f88d382eda0>, <generic.ugraph.UGraph object at 0x7f88d382edd8>, <generic.ugraph.UGraph object at 0x7f88d382ee10>, <generic.ugraph.UGraph object at 0x7f88d382ee48>, <generic.ugraph.UGraph object at 0x7f88d382ee80>, <generic.ugraph.UGraph object at 0x7f88d382eeb8>, <generic.ugraph.UGraph object at 0x7f88d382eef0>, <generic.ugraph.UGraph object at 0x7f88d382ef28>, <generic.ugraph.UGraph object at 0x7f88d382ef60>, <generic.ugraph.UGraph object at 0x7f88d382ef98>, <generic.ugraph.UGraph object at 0x7f88d382efd0>, <generic.ugraph.UGraph object at 0x7f88d382f048>, <generic.ugraph.UGraph object at 0x7f88d382f080>, <generic.ugraph.UGraph object at 0x7f88d382f0b8>, <generic.ugraph.UGraph object at 0x7f88d382f0f0>, <generic.ugraph.UGraph object at 0x7f88d382f128>, <generic.ugraph.UGraph object at 0x7f88d382f160>, <generic.ugraph.UGraph object at 0x7f88d382f198>, <generic.ugraph.UGraph object at 0x7f88d382f1d0>, <generic.ugraph.UGraph object at 0x7f88d382f208>, <generic.ugraph.UGraph object at 0x7f88d382f240>, <generic.ugraph.UGraph object at 0x7f88d382f278>, <generic.ugraph.UGraph object at 0x7f88d382f2b0>, <generic.ugraph.UGraph object at 0x7f88d382f2e8>, <generic.ugraph.UGraph object at 0x7f88d382f320>, <generic.ugraph.UGraph object at 0x7f88d3914e10>, <generic.ugraph.UGraph object at 0x7f88d3914d30>, <generic.ugraph.UGraph object at 0x7f88d38a6128>, <generic.ugraph.UGraph object at 0x7f88d38a6780>, <generic.ugraph.UGraph object at 0x7f88d389c080>, <generic.ugraph.UGraph object at 0x7f88d38bb128>, <generic.ugraph.UGraph object at 0x7f88d38bb6a0>, <generic.ugraph.UGraph object at 0x7f88d38bb400>, <generic.ugraph.UGraph object at 0x7f88d38bb550>, <generic.ugraph.UGraph object at 0x7f88d38bb518>, <generic.ugraph.UGraph object at 0x7f88d38bb0f0>, <generic.ugraph.UGraph object at 0x7f88d38bb208>, <generic.ugraph.UGraph object at 0x7f88d38bb2b0>, <generic.ugraph.UGraph object at 0x7f88d38bba90>, <generic.ugraph.UGraph object at 0x7f88d38bb160>, <generic.ugraph.UGraph object at 0x7f88d38bb5c0>, <generic.ugraph.UGraph object at 0x7f88d38bb390>, <generic.ugraph.UGraph object at 0x7f88d38bb4e0>, <generic.ugraph.UGraph object at 0x7f88d38bbe80>, <generic.ugraph.UGraph object at 0x7f88d38bb588>, <generic.ugraph.UGraph object at 0x7f88d38bbdd8>, <generic.ugraph.UGraph object at 0x7f88

## eigen

In [7]:
from h2py.eigen import EigenValueBruteForce

In [8]:
graphs = list(UniqueCertificates("../../data/db_unique_certificates.csv"))

graphs_i = [graph for graph in graphs if EigenValueBruteForce.compute(graph).dtype == np.complex128]
graphs_i

[]

In [33]:
aa.dtype

dtype('complex128')

In [18]:
aa = EigenValueBruteForce().compute(graphs_i[0])
aa

array([ 3.37228132e+00+0.00000000e+00j, -2.37228132e+00+0.00000000e+00j,
       -8.17181493e-17+0.00000000e+00j, -1.09694870e-16+4.12678875e-17j,
       -1.09694870e-16-4.12678875e-17j, -1.00000000e+00+0.00000000e+00j])

In [31]:
aa.real

array([ 3.37228132e+00, -2.37228132e+00, -8.17181493e-17, -1.09694870e-16,
       -1.09694870e-16, -1.00000000e+00])

In [30]:
np.sum(np.absolute(aa.imag) > 1e-10)

0

In [23]:
aa[aa.imag]

array([-1.0969487e-16+4.12678875e-17j])

In [20]:
np.absolute(aa)

array([3.37228132e+00, 2.37228132e+00, 8.17181493e-17, 1.17200695e-16,
       1.17200695e-16, 1.00000000e+00])

In [11]:
graphs = list(UniqueCertificates("../../data/db_unique_certificates.csv"))

[graph for graph in graphs]

{dtype('float64'), dtype('complex128')}

# scratchyard